In [135]:
xlim,ylim = 3,2
import copy
class GameState:
    def __init__(self):
        """
        Attributes
        ----------
        _parity: bool
            keep track of player turn.
        
        _player_loc: list(tuple)
            keep track of player current position
            
        _board: list(list)
            Board representation
        """
        self._parity = 0
        self._player_location = [None,None]
        self._board = [[0]*ylim for _ in range(xlim)]
        self._board[-1][-1] = 1
    
    
    def get_legal_moves(self):
        """
        Return a list of all legal moves available for current player.
        """
        loc = self._player_location[self._parity]
        if not loc:
            return self._get_blank_spaces()
        moves = []
        path = [(-1,-1),(-1,0),(-1,1),(0,-1),
                (0,1),(1,-1),(1,0),(1,1)]
        for i,j in path:
            x,y = loc
            while 0 <= x+i < xlim and 0 <= y+j < ylim:
                x,y = x+i,y+j
                if self._board[x][y]:
                    break
                moves.append((x,y))
        return moves
        
    
    def _get_blank_spaces(self):
        """
        Return a list of blank spaces.
        """
        return [(x,y) for x in range(xlim) for y in range(ylim) 
                if self._board[x][y] == 0]
        
        
    def forecast_move(self,move):
        """
        Return a new board with specified move applied to the current player
        
        Parameters
        ----------
        move: tuple
            The target position player want to move.
        """
        if move not in self.get_legal_moves():
            raise RuntimeError("illegal move")
        newBoard = copy.deepcopy(self)
        newBoard._board[move[0]][move[1]] = 1
        newBoard._player_location[self._parity] = move
        newBoard._parity ^= 1
        return newBoard

In [136]:
print("Creating empty game board...")
g = GameState()


Creating empty game board...


In [137]:
print("Getting legal moves for player 1...")
p1_empty_moves = g.get_legal_moves()


Getting legal moves for player 1...


In [138]:
print("Found {} legal moves.".format(len(p1_empty_moves or [])))

Found 5 legal moves.


In [139]:
print("Applying move (0, 0) for player 1...")
g1 = g.forecast_move((0, 0))

Applying move (0, 0) for player 1...


In [140]:
xlim, ylim = 3, 2  # board dimensions
import copy
class GameState1:
    """
    Attributes
    ----------
    _board: list(list)
        Represent the board with a 2d array _board[x][y]
        where open spaces are 0 and closed spaces are 1
    
    _parity: bool
        Keep track of active player initiative (which
        player has control to move) where 0 indicates that
        player one has initiative and 1 indicates player 2
    
    _player_locations: list(tuple)
        Keep track of the current location of each player
        on the board where position is encoded by the
        board indices of their last move, e.g., [(0, 0), (1, 0)]
        means player 1 is at (0, 0) and player 2 is at (1, 0)
    
    """

    def __init__(self):
        self._board = [[0] * ylim for _ in range(xlim)]
        self._board[-1][-1] = 1  # block lower-right corner
        self._parity = 0
        self._player_locations = [None, None]

    def forecast_move(self, move):
        """ Return a new board object with the specified move
        applied to the current game state.
        
        Parameters
        ----------
        move: tuple
            The target position for the active player's next move
        """
        print(move)
        if move not in self.get_legal_moves():
            raise RuntimeError("Attempted forecast of illegal move")
        newBoard = copy.deepcopy(self)
        newBoard._board[move[0]][move[1]] = 1
        newBoard._player_locations[self._parity] = move
        newBoard._parity ^= 1
        return newBoard

    def get_legal_moves(self):
        """ Return a list of all legal moves available to the
        active player.  Each player should get a list of all
        empty spaces on the board on their first move, and
        otherwise they should get a list of all open spaces
        in a straight line along any row, column or diagonal
        from their current position. (Players CANNOT move
        through obstacles or blocked squares.)
        """
        loc = self._player_locations[self._parity]
        if not loc:
            return self._get_blank_spaces()
        moves = []
        rays = [(1, 0), (1, -1), (0, -1), (-1, -1),
                (-1, 0), (-1, 1), (0, 1), (1, 1)]
        for dx, dy in rays:
            _x, _y = loc
            while 0 <= _x + dx < xlim and 0 <= _y + dy < ylim:
                _x, _y = _x + dx, _y + dy
                if self._board[_x][_y]:
                    break
                moves.append((_x, _y))
        return moves

    def _get_blank_spaces(self):
        """ Return a list of blank spaces on the board."""
        return [(x, y) for y in range(ylim) for x in range(xlim)
                if self._board[x][y] == 0]

In [143]:
print("Getting legal moves for player 2...")
p2_empty_moves = g1.get_legal_moves()
print(p2_empty_moves)

Getting legal moves for player 2...
[(0, 1), (1, 0), (1, 1), (2, 0)]


In [142]:
if (0, 0) in set(p2_empty_moves):
    print("Failed\n  Uh oh! (0, 0) was not blocked properly when " +
          "player 1 moved there.")
else:
    print("Everything looks good!")

Everything looks good!
